In [11]:
import pandas as pd
import json
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
DATA_DIR = PROJECT_ROOT / "data" / "processed"

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [12]:
df = pd.read_parquet(DATA_DIR / "analysis_ready.parquet")
print(f"Shape: {df.shape}")
df.head()

Shape: (18, 19)


,utility_id_ferc1,utility_id_eia,utility_name,report_year,om_production,om_transmission,om_distribution,om_customer_service,om_admin_general,om_other,om_total,rate_base,sales_mwh_total,customers_total,revenue_total_k,om_per_customer,om_per_mwh,rate_base_per_customer,revenue_per_customer
0,206,14328,PG&E,2018,6.661930e+08,137056448.0,83799264.0,43312204.0,77286160.0,4.426125e+09,5.433773e+09,6.245131e+09,49222914.0,3847071.0,10175681.0,1412.444050,110.391119,1623.347025,2645.046322
1,206,14328,PG&E,2019,5.957743e+08,118375840.0,90316728.0,45180432.0,74446440.0,4.136594e+09,5.060687e+09,6.477699e+09,36462157.0,2635291.0,7885748.0,1920.352386,138.792868,2458.058359,2992.363272
2,206,14328,PG&E,2020,5.500829e+08,134712592.0,89512896.0,38954164.0,72012632.0,3.836941e+09,4.722216e+09,6.545996e+09,36233951.0,2463331.0,8351402.0,1917.004433,130.325739,2657.375633,3390.288191
3,206,14328,PG&E,2021,1.137121e+09,158427280.0,87704312.0,38020776.0,86096584.0,6.796977e+09,8.304347e+09,7.049240e+09,33924626.0,2269282.0,8018395.0,3659.460189,244.788171,3106.374644,3533.450228
4,206,14328,PG&E,2022,7.757265e+08,170676976.0,97418920.0,45000552.0,86999696.0,5.846398e+09,7.022221e+09,7.965495e+09,31146948.0,2119635.0,8834188.0,3312.938931,225.454555,3757.956108,4167.787378


In [13]:
with open(DATA_DIR / "analysis_results.json", "r") as f:
    results = json.load(f)

print(results.keys())

dict_keys(['trend_results', 'regression_summary', 'vif', 'outliers', 'benchmarks', 'yoy_changes', 'utility_summaries'])


## REVIEW Historical Patterns, Drivers, Peer Performance
Check Analyses relevant to rate case filings
1. Trend Analysis
2. Cost Driver Regression
3. Outlier Detection
4. Peer Benchmarking
5. YoY change
6. Summary Statistics


# 1. Trend Analysis
### Purpose
- Detect whether proposed costs are an extrapolation of historical trends or an unjustified jump
- Check slope, CAGR, variability of costs over time

### Questions **
- " Is 8.3% consistent with SCE's historical growth -- Trend Analysis shows SCE's 5-year CAGR; Request exceeds trend


In [14]:
pd.DataFrame(results["trend_results"])


,PG&E,SDG&E,SCE
utility_id,206,241,252
metric,om_total,om_total,om_total
slope,224508957.257143,36072658.285714,16117340.571429
intercept,-447677809386.05719,-72655029218.285721,-32455188600.57143
r_squared,0.088567,0.679876,0.679211
p_value,0.566775,0.043476,0.043669
cagr_percent,-1.21348,17.406144,14.598539
mean,5942538752.0,229776848.0,109898024.0
std,1411336832.0,81845976.0,36586844.0
coefficient_of_variation,23.749729,35.619766,33.29163


# 2. Cost Driver Regression Summary
##Purpose
- Tests whether cost increases are explained by legitimate drivers (more customers, more infrastructure) or unexplained
- Models O&M as a function of customers, sales, and rate base
- Statistically significant coefficients = defensible cost growth

### Example
- "Is the increase explained by growth in customers or load?" -- Regression shows customers and rate base explains 87% of O&M variance, but SCE customer growth only 1.2%. Gap remains unexplained from the regression

In [15]:
print(results["regression_summary"])


                            OLS Regression Results                            
Dep. Variable:               om_total   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     71.34
Date:                Mon, 08 Dec 2025   Prob (F-statistic):           1.01e-08
Time:                        17:37:14   Log-Likelihood:                -392.12
No. Observations:                  18   AIC:                             792.2
Df Residuals:                      14   BIC:                             795.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            2.891e+08   6.83e+08     

# 3. Peer Benchmarking
### Purpose
- "Is this utility expensive relative to peers?" 
- 90th percentile on O&M per customer needs to justify why.

### Question **
- "Is SCE expensive compared to PG&E and SDG&E?" -- Peer benchmark shows SCE's O&M per customer is 12% above per average

In [16]:
pd.DataFrame(results["benchmarks"])


,utility_id_ferc1,utility_name,om_per_customer,z_score,percentile,rank
0,206,PG&E,2409.550332,1.103489,100.000000,1
1,241,SDG&E,762.143678,-0.257240,66.666667,2
2,252,SCE,49.040943,-0.846250,33.333333,3


# 4. Year-Over-Year Changes
Calculates percent change between consecutive years
### Purpose
- Spots sudden spikes or drops
- difference in Year-over-year changes amongst peers are what we are looking for

### Implementation and Use Cases
- Staff calculates YoY percent change for major cost categories
- Tracking post-rate-case performance (did costs stay within authorized levels?)

In [17]:
pd.DataFrame(results["yoy_changes"])


,utility_id_ferc1,utility_name,report_year,metric_value,yoy_pct_change
0,206,PG&E,2018,5.433773e+09,NaN
1,206,PG&E,2019,5.060687e+09,-6.866044
2,206,PG&E,2020,4.722216e+09,-6.688240
3,206,PG&E,2021,8.304347e+09,75.856979
4,206,PG&E,2022,7.022221e+09,-15.439213
5,206,PG&E,2023,5.111989e+09,-27.202684
6,241,SDG&E,2018,1.751416e+08,NaN
7,241,SDG&E,2019,1.785091e+08,1.922751
8,241,SDG&E,2020,1.928683e+08,8.043932
9,241,SDG&E,2021,2.055899e+08,6.596004


# Outlier Detection
- Flags utility-years with abnormal cost metrics (z-score > 2)
### Purpose
- Identifies what years/utilities may require deeper investigation
- Outlier isn't automatically bad, (EX: COVID years, wildfire costs, major capital projects) but it demands explanation.

### Use Cases and Implementation
- Understand anomalies that shouldn't anchor baselin calcs
- Isolating one-time events (wildfires, storms, Covid)
- Flag data errors in filings
- Determining whether to normalize test data
- "Were there unusual years affecting the baseline?" -- Outlier detection will have flagged 2020 (probably COVID) and 2021 (wildfire costs possible)

In [18]:
pd.DataFrame(results["outliers"])

,utility_id_ferc1,utility_id_eia,utility_name,report_year,om_production,om_transmission,om_distribution,om_customer_service,om_admin_general,om_other,...,rate_base,sales_mwh_total,customers_total,revenue_total_k,om_per_customer,om_per_mwh,rate_base_per_customer,revenue_per_customer,z_score,is_outlier
0,206,14328,PG&E,2021,1.137121e+09,158427280.0,87704312.0,38020776.0,86096584.0,6.796977e+09,...,7.049240e+09,33924626.0,2269282.0,8018395.0,3659.460189,244.788171,3106.374644,3533.450228,2.250369,True


# Summary Utility Statistics
- Aggregates mean, std, min, max, CAGR by utility
### Purpose
- Provides a digestible utility profile for each utility for decision making
- Practice summarizing simply for commissioners/shareholders
- Compare utility-specific trends at a glance
### Implementation + Use Cases **
- Prep 1-shot utility profile: avg O&M, customer count, rate base, growth rate
- Prep for full case filing analsis
- References during meetings
  

In [19]:
pd.DataFrame(results["utility_summaries"])

,utility_id_ferc1,utility_name,metric,mean,std,min,max,cagr_percent,n_years
0,206,PG&E,om_total,5.942539e+09,1.411337e+09,4.722216e+09,8.304347e+09,-1.213479,5
1,206,PG&E,om_per_customer,2.438625e+03,8.774338e+02,1.412444e+03,3.659460e+03,11.273808,5
2,206,PG&E,om_per_mwh,1.734480e+02,5.502107e+01,1.103911e+02,2.447882e+02,11.581058,5
3,206,PG&E,rate_base,7.060731e+09,7.911268e+08,6.245131e+09,8.080827e+09,5.288959,5
4,241,SDG&E,om_total,2.297768e+08,8.184598e+07,1.751416e+08,3.907000e+08,17.406141,5
5,241,SDG&E,om_per_customer,2.572289e+02,2.526977e+02,1.212668e+02,7.621437e+02,44.430533,5
6,241,SDG&E,om_per_mwh,2.839437e+01,2.838319e+01,1.156889e+01,8.457682e+01,48.864765,5
7,241,SDG&E,rate_base,4.380557e+08,8.169430e+07,3.396222e+08,5.577683e+08,10.431135,5
8,252,SCE,om_total,1.098980e+08,3.658684e+07,8.398890e+07,1.790673e+08,14.598536,5
9,252,SCE,om_per_customer,2.779219e+01,1.144793e+01,1.827948e+01,4.904094e+01,21.820108,5
